In [151]:
# HAC: heterroskedasticity and autocorrelation corrected standard errors
# https://www.youtube.com/watch?v=hKyg1pGDmj0
import itertools
import numpy as np
import pandas as pd
import scipy.stats as stats

In [98]:
path = "/Users/weizhang/Documents/_GIT/quant-strategies/data/HAC.csv"
data = pd.read_csv(path)
data.head()

,Dates,Exxon Mobil,Constant,S&P 500,Oil
0,1/3/2022,0.038405,1,0.005790,0.011598
1,1/4/2022,0.037614,1,-0.000335,0.011912
2,1/5/2022,0.012437,1,-0.019202,0.011183
3,1/6/2022,0.023521,1,-0.000939,0.020664
4,1/7/2022,0.008197,1,-0.003953,-0.006986


In [100]:
X = data[["Constant", "S&P 500", "Oil"]]
y = data["Exxon Mobil"]

In [116]:
coeff = np.matmul(np.matmul(np.linalg.inv(np.matmul(np.transpose(X), X)), np.transpose(X)), y)

In [104]:
data["Expected"] = np.matmul(X.to_numpy(), coeff)
data["Residual"] = data["Exxon Mobil"] - data["Expected"]

In [110]:
T = X.shape[0]
k = X.shape[1]
df = T - k
error = np.sqrt(np.sum(data["Residual"] * data["Residual"]) / df)
variance = error ** 2 * np.linalg.inv(np.matmul(np.transpose(X), X))

In [115]:
OLS = pd.DataFrame()
OLS["Coeff"] = coeff
OLS.index = ["Constant", "S&P 500", "Oil"]
OLS["S.E."] = [np.sqrt(variance[0][0]), np.sqrt(variance[1][1]), np.sqrt(variance[2][2])]
OLS["t-stat"] = OLS["Coeff"] / OLS["S.E."]
OLS["p-value"] = [stats.t.sf(np.abs(OLS.loc["Constant", "t-stat"]), df) * 2,
                  stats.t.sf(np.abs(OLS.loc["S&P 500", "t-stat"]), df) * 2,
                  stats.t.sf(np.abs(OLS.loc["Oil", "t-stat"]), df) * 2]
OLS.head()

,Coeff,S.E.,t-stat,p-value
Constant,0.002959,0.002273,1.301674,1.980874e-01
S&P 500,0.007967,0.172776,0.046112,9.633771e-01
Oil,0.385618,0.063753,6.048659,1.073621e-07


In [158]:
lags = 3
lags_formula = 4 * (T / 100) ** (2 / 9)

In [160]:
weights = np.multiply(np.matmul(data[["Residual"]], np.transpose(data[["Residual"]])), np.array([max(1 - np.abs(i - j) / (lags + 1), 0) for i, j in itertools.product(np.arange(T), np.arange(T))]).reshape((T, T)))

In [166]:
np.matmul(np.matmul(np.linalg.inv(np.matmul(np.transpose(X), X)), np.matmul(np.matmul(np.transpose(X), weights), X)), np.linalg.inv(np.matmul(np.transpose(X), X)))

,0,1,2
0,0.000005,0.000084,-0.000018
1,0.000084,0.020865,-0.000563
2,-0.000018,-0.000563,0.002824
